<a href="https://colab.research.google.com/github/elephanti/NLPProject2024/blob/main/Train_and_Test_with_augmented_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/elephanti/NLPProject2024.git

Cloning into 'NLPProject2024'...
remote: Enumerating objects: 1078, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 1078 (delta 3), reused 1 (delta 1), pack-reused 1072 (from 1)
Receiving objects: 100% (1078/1078), 32.75 MiB | 16.00 MiB/s, done.
Resolving deltas: 100% (421/421), done.


In [2]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip -d NLPProject2024/classifiers/embeddings

--2024-08-15 00:01:27--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-08-15 00:01:27--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-08-15 00:01:28--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [3]:
!pip install tensorflow==2.15.1
!pip install ktrain


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 73.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 113.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.0 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.16.0
    Uninstalling wrapt-1.16.0:
      Successfully uninstalled wrapt-1.16.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.0
    Uninstalling ml-dtypes-0.4.0:
      Successfully uninstalled ml-dtypes-0.4.0
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorboard
    Found existing installatio

In [4]:
!pip install transformers==4.37.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 98.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4


In [5]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder


def load_data(full_train_path, generated_train_path=None, test_path=None, val_path=None):
    print(f"\tFull train path: {full_train_path}")
    print(f"\tGenerated train path: {generated_train_path}")
    print(f"\tTest path: {test_path}")
    train_df = pd.read_csv(full_train_path)

    if generated_train_path:
        train_df = pd.concat([train_df, pd.read_csv(generated_train_path)], ignore_index=True)

    train_text = train_df['text'].tolist()
    train_label = train_df['label']

    le = LabelEncoder()
    y_train = le.fit_transform(train_label)

    test_df = pd.read_csv(test_path)
    test_text = test_df['text'].tolist()
    test_label = test_df['label']
    y_test = le.transform(test_label)

    if val_path:
      val_df = pd.read_csv(val_path)
      val_text = val_df['text'].tolist()
      val_label = val_df['label']
      y_val = le.transform(val_label)
      return train_text, y_train, test_text, y_test, le, val_text, y_val

    return train_text, y_train, test_text, y_test, le

In [6]:
import pandas as pd
import glob
import os
import joblib
from NLPProject2024.classifiers.bert_ktrain import DistilBERT
from NLPProject2024.classifiers.lstm_glove import LSTMGlove
from NLPProject2024.classifiers.svm_glove import SVMGlove
from NLPProject2024.classifiers.svm_tfidf import SVMTFIDF
import time

class ModelTrainer:
    def __init__(self, classifier_name, glove_file, num_labels):
        self.classifier = None
        self.classifier_name = classifier_name
        self.glove_file = glove_file
        self.num_labels = num_labels

    def train_classifier(self, X_train, y_train, model_path, X_test=None, y_test=None, X_val=None, y_val=None):
        print(f"Training {self.classifier_name}...")
        start_time = time.time()
        if self.classifier_name == 'lstm_glove':
            self.classifier = LSTMGlove
            clf_instance = self.classifier(self.glove_file, self.num_labels)
        elif self.classifier_name == 'svm_glove':
            self.classifier = SVMGlove
            clf_instance = self.classifier(self.glove_file)
        elif self.classifier_name == 'bert_ktrain':
            self.classifier = DistilBERT
            clf_instance = self.classifier(maxlen=50, batch_size=10, learning_rate=5e-5, epochs=20,
                                           early_stopping=4, reduce_on_plateau=2)
        else:
            self.classifier = SVMTFIDF
            clf_instance = self.classifier()

        clf_instance.train(X_train, y_train, X_val, y_val)

        accuracy, precision, recall, f1 = clf_instance.evaluate(X_test, y_test)
        if self.classifier_name in ['lstm_glove', 'bert_ktrain']:
            results = {"classifier": self.classifier_name,
                       "test_accuracy": round(accuracy.item(), 4),
                       "precision": round(precision.item(), 4),
                       "recall": round(recall.item(), 4),
                       "f1_score": round(f1.item(), 4)}
        else:
            results = {"classifier": self.classifier_name,
                       "test_accuracy": round(accuracy, 4),
                       "precision": round(precision, 4),
                       "recall": round(recall, 4),
                       "f1_score": round(f1, 4)}

        clf_instance.save(f'{model_path}/{self.classifier_name}')
        end_time = time.time()
        results["training_time"] = round(end_time - start_time, 2)

        return results


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
df_result = pd.DataFrame(columns=['dataset', 'model', 'test_accuracy', 'precision', 'recall', 'f1_score'])
glove_file = 'NLPProject2024/classifiers/embeddings/glove.6B.100d.txt'

classifiers = {
    'svm_tfidf': SVMTFIDF,
    'svm_glove': SVMGlove,
    'lstm_glove': LSTMGlove,
    'bert_ktrain': DistilBERT
}

lambadas = ['Lambada', 'Lambada+', 'Lambada+Instruct']
llms = ['Llama3', 'Mistral', 'GPT2']
dataset_names = ['ATIS', 'TREC']
subset_sizes = [5, 10, 20, 50, 100]

In [11]:
full_df_result = pd.DataFrame(columns=['dataset', 'model', 'test_accuracy', 'precision', 'recall', 'f1_score'])

# Train on full training data
for name in dataset_names:
    X_train, y_train, X_test, y_test, le, X_val, y_val = load_data(f'NLPProject2024/datasets/{name}/{name.lower()}.train.csv',
                                                                   test_path=f'NLPProject2024/datasets/{name}/{name.lower()}.test.csv',
                                                                   val_path=f'NLPProject2024/datasets/{name}/{name.lower()}.valid.csv')
    model_path = f'final_results/models/{name.lower()}_train'
    os.makedirs(model_path, exist_ok=True)

    joblib.dump(le, f'{model_path}/label_encoder.pkl')

    for classifier in classifiers:
        print(f"Training {name} with {classifier}...")
        trainer = ModelTrainer(classifier, glove_file, len(le.classes_))
        train_results = trainer.train_classifier(X_train, y_train, model_path, X_test=X_test, y_test=y_test, X_val=X_val, y_val=y_val)

        train_results["dataset"] = f'{name.lower()}_full'
        full_df_result = pd.concat([full_df_result, pd.DataFrame([train_results])], ignore_index=True)
        print(f"Finished training {name} with {classifier}")
        print(train_results)
        print("-" * 50)

	Full train path: NLPProject2024/datasets/ATIS/atis.train.csv
	Generated train path: None
	Test path: NLPProject2024/datasets/ATIS/atis.test.csv
Training ATIS with svm_tfidf...
Training svm_tfidf...

Validation Results:
Accuracy: 0.9743
Precision: 0.9704
Recall: 0.9743
F1 Score: 0.9711


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.9588
Precision: 0.9516
Recall: 0.9588
F1 Score: 0.9532
Model saved.
Finished training ATIS with svm_tfidf
{'classifier': 'svm_tfidf', 'test_accuracy': 0.9588, 'precision': 0.9516, 'recall': 0.9588, 'f1_score': 0.9532, 'training_time': 6.85, 'dataset': 'atis_full'}
--------------------------------------------------
Training ATIS with svm_glove...
Training svm_glove...

Validation Results:
Accuracy: 0.9383
Precision: 0.9332
Recall: 0.9383
F1 Score: 0.9349


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.9280
Precision: 0.9121
Recall: 0.9280
F1 Score: 0.9170
Model saved.
Finished training ATIS with svm_glove
{'classifier': 'svm_glove', 'test_accuracy': 0.928, 'precision': 0.9121, 'recall': 0.928, 'f1_score': 0.917, 'training_time': 11.49, 'dataset': 'atis_full'}
--------------------------------------------------
Training ATIS with lstm_glove...
Training lstm_glove...
Epoch 1/20, Loss: 0.9508, Accuracy: 0.7714
Accuracy: 0.9108
Precision: 0.8858
Recall: 0.9108
F1 Score: 0.8922
Validation - Accuracy: 0.9108, Precision: 0.8858, Recall: 0.9108, F1 Score: 0.8922


...

Accuracy: 0.9680
Precision: 0.9679
Recall: 0.9680
F1 Score: 0.9671
Validation - Accuracy: 0.9680, Precision: 0.9679, Recall: 0.9680, F1 Score: 0.9671
Epoch 20/20, Loss: 0.0911, Accuracy: 0.9730


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.9611
Precision: 0.9615
Recall: 0.9611
F1 Score: 0.9601
Validation - Accuracy: 0.9611, Precision: 0.9615, Recall: 0.9611, F1 Score: 0.9601
Accuracy: 0.9599
Precision: 0.9616
Recall: 0.9599
F1 Score: 0.9591
Model and embeddings saved.
Finished training TREC with lstm_glove
{'classifier': 'lstm_glove', 'test_accuracy': 0.9599, 'precision': 0.9616, 'recall': 0.9599, 'f1_score': 0.9591, 'training_time': 112.23, 'dataset': 'trec_full'}
--------------------------------------------------
Training TREC with bert_ktrain...
Training bert_ktrain...
preprocessing train...
language: en
train sequence lengths:
	mean : 10
	95percentile : 17
	99percentile : 22


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 10
	95percentile : 17
	99percentile : 21




begin training using triangular learning rate policy with max lr of 5e-05...
Epoch 1/20
1277/1277 [==============================] - 74s 50ms/step - loss: 0.8433 - accuracy: 0.8130 - val_loss: 0.2156 - val_accuracy: 0.9524
Epoch 2/20
1277/1277 [==============================] - 62s 48ms/step - loss: 0.1225 - accuracy: 0.9745 - val_loss: 0.1396 - val_accuracy: 0.9755
Epoch 3/20
1277/1277 [==============================] - 63s 48ms/step - loss: 0.0671 - accuracy: 0.9859 - val_loss: 0.0941 - val_accuracy: 0.9806
Epoch 4/20
1277/1277 [==============================] - 63s 48ms/step - loss: 0.0372 - accuracy: 0.9915 - val_loss: 0.0798 - val_accuracy: 0.9831
Epoch 5/20
1277/1277 [==============================] - 63s 48ms/step - loss: 0.0317 - accuracy: 0.9933 - val_loss: 0.0791 - val_accuracy: 0.9837
Epoch 6/20
1277/1277 [==============================] - 62s 48ms/step - loss: 0.0273 - accuracy: 0.9931 - val_loss: 0.0898 - val_accuracy: 0.9856
Epoch 7/20
1277/1277 [=======================

In [ ]:
full_df_result.head(10)

,dataset,model,test_accuracy,precision,recall,f1_score,classifier,training_time
0,atis_full,NaN,0.9588,0.9516,0.9588,0.9532,svm_tfidf,6.87
1,atis_full,NaN,0.9280,0.9121,0.9280,0.9170,svm_glove,11.51
2,atis_full,NaN,0.9811,0.9811,0.9811,0.9805,lstm_glove,60.87
3,atis_full,NaN,0.9897,0.9888,0.9897,0.9891,bert_ktrain,262.77
4,trec_full,NaN,0.9599,0.9611,0.9599,0.9598,svm_tfidf,81.70
5,trec_full,NaN,0.8177,0.8224,0.8177,0.8168,svm_glove,40.77
6,trec_full,NaN,0.9630,0.9649,0.9630,0.9631,lstm_glove,109.84
7,trec_full,NaN,0.9831,0.9838,0.9831,0.9831,bert_ktrain,875.90


In [9]:
subsets_df_result = pd.DataFrame(columns=['dataset', 'classifier', 'model', 'subset_size', 'method', 'test_accuracy',
                                          'precision', 'recall', 'f1_score'])

# Train on the subsets
for name in dataset_names:
    for subset_size in subset_sizes:
        subset_path = f'NLPProject2024/datasets/{name}/sampled_subsets/ver1/{name.lower()}_{subset_size}_subset.csv'

        # subset only training
        X_train, y_train, X_test, y_test, le, X_val, y_val = load_data(subset_path,
                                                                       test_path=f'NLPProject2024/datasets/{name}/{name.lower()}.test.csv',
                                                                       val_path=f'NLPProject2024/datasets/{name}/{name.lower()}.valid.csv')
        model_path = f'final_results/models/{name.lower()}_{subset_size}_subset_train'
        os.makedirs(model_path, exist_ok=True)

        joblib.dump(le, f'{model_path}/label_encoder.pkl')

        for classifier in classifiers:
            print(f"Training {name} with {classifier} and subset size {subset_size}...")
            trainer = ModelTrainer(classifier, glove_file, len(le.classes_))
            train_results = trainer.train_classifier(X_train, y_train, model_path, X_test=X_test, y_test=y_test, X_val=X_val, y_val=y_val)

            train_results["dataset"] = f'{name.lower()}_{subset_size}_subset_train'
            train_results["method"] = "Baseline"
            train_results["model"] = None
            train_results["subset_size"] = subset_size
            subsets_df_result = pd.concat([subsets_df_result, pd.DataFrame([train_results])], ignore_index=True)
            print(f"Finished training {name} with {classifier} and subset size {subset_size}")
            print(train_results)
            print("-" * 50)

	Full train path: NLPProject2024/datasets/ATIS/sampled_subsets/ver1/atis_5_subset.csv
	Generated train path: None
	Test path: NLPProject2024/datasets/ATIS/atis.test.csv
Training ATIS with svm_tfidf and subset size 5...
Training svm_tfidf...

Validation Results:
Accuracy: 0.5798
Precision: 0.8624
Recall: 0.5798
F1 Score: 0.6576
Accuracy: 0.6055
Precision: 0.8235
Recall: 0.6055
F1 Score: 0.6602
Model saved.
Finished training ATIS with svm_tfidf and subset size 5
{'classifier': 'svm_tfidf', 'test_accuracy': 0.6055, 'precision': 0.8235, 'recall': 0.6055, 'f1_score': 0.6602, 'training_time': 0.09, 'dataset': 'atis_5_subset_train', 'method': 'Baseline', 'model': None, 'subset_size': 5}
--------------------------------------------------
Training ATIS with svm_glove and subset size 5...
Training svm_glove...


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Validation Results:
Accuracy: 0.5832
Precision: 0.8216
Recall: 0.5832
F1 Score: 0.6616
Accuracy: 0.5815
Precision: 0.7998
Recall: 0.5815
F1 Score: 0.6436
Model saved.


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Finished training ATIS with svm_glove and subset size 5
{'classifier': 'svm_glove', 'test_accuracy': 0.5815, 'precision': 0.7998, 'recall': 0.5815, 'f1_score': 0.6436, 'training_time': 8.55, 'dataset': 'atis_5_subset_train', 'method': 'Baseline', 'model': None, 'subset_size': 5}
--------------------------------------------------
Training ATIS with lstm_glove and subset size 5...
Training lstm_glove...
Epoch 1/20, Loss: 3.1529, Accuracy: 0.0488
Accuracy: 0.0669
Precision: 0.0141
Recall: 0.0669
F1 Score: 0.0217
Validation - Accuracy: 0.0669, Precision: 0.0141, Recall: 0.0669, F1 Score: 0.0217


...

Accuracy: 0.7774
Precision: 0.8269
Recall: 0.7774
F1 Score: 0.7778
Validation - Accuracy: 0.7774, Precision: 0.8269, Recall: 0.7774, F1 Score: 0.7778
Epoch 20/20, Loss: 0.2328, Accuracy: 0.9319


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.7981
Precision: 0.8400
Recall: 0.7981
F1 Score: 0.8000
Validation - Accuracy: 0.7981, Precision: 0.8400, Recall: 0.7981, F1 Score: 0.8000
Accuracy: 0.7970
Precision: 0.8369
Recall: 0.7970
F1 Score: 0.7980
Model and embeddings saved.
Finished training TREC with lstm_glove and subset size 100
{'classifier': 'lstm_glove', 'test_accuracy': 0.797, 'precision': 0.8369, 'recall': 0.797, 'f1_score': 0.798, 'training_time': 61.49, 'dataset': 'trec_100_subset_train', 'method': 'Baseline', 'model': None, 'subset_size': 100}
--------------------------------------------------
Training TREC with bert_ktrain and subset size 100...
Training bert_ktrain...
preprocessing train...
language: en
train sequence lengths:
	mean : 10
	95percentile : 16
	99percentile : 20


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 10
	95percentile : 17
	99percentile : 21




begin training using triangular learning rate policy with max lr of 5e-05...
Epoch 1/20
363/363 [==============================] - 29s 57ms/step - loss: 1.7204 - accuracy: 0.6346 - val_loss: 0.6500 - val_accuracy: 0.8451
Epoch 2/20
363/363 [==============================] - 19s 51ms/step - loss: 0.3228 - accuracy: 0.9352 - val_loss: 0.3817 - val_accuracy: 0.9022
Epoch 3/20
363/363 [==============================] - 18s 51ms/step - loss: 0.1372 - accuracy: 0.9769 - val_loss: 0.4523 - val_accuracy: 0.8953
Epoch 4/20
363/363 [==============================] - 19s 51ms/step - loss: 0.0742 - accuracy: 0.9893 - val_loss: 0.3069 - val_accuracy: 0.9254
Epoch 5/20
363/363 [==============================] - 18s 51ms/step - loss: 0.0433 - accuracy: 0.9931 - val_loss: 0.3761 - val_accuracy: 0.9135
Epoch 6/20
363/363 [==============================] - ETA: 0s - loss: 0.0341 - accuracy: 0.9939
Epoch 00006: Reducing Max LR on Plateau: new max lr will be 2.5e-05 (if not early_stopping).
363/363 [===

In [10]:
subsets_df_result.head(20)

,dataset,classifier,model,subset_size,method,test_accuracy,precision,recall,f1_score,training_time
0,atis_5_subset_train,svm_tfidf,None,5,Baseline,0.6055,0.8235,0.6055,0.6602,0.09
1,atis_5_subset_train,svm_glove,None,5,Baseline,0.5815,0.7998,0.5815,0.6436,8.55
2,atis_5_subset_train,lstm_glove,None,5,Baseline,0.4889,0.7185,0.4889,0.5588,39.81
3,atis_5_subset_train,bert_ktrain,None,5,Baseline,0.8456,0.8981,0.8456,0.8620,50.93
4,atis_10_subset_train,svm_tfidf,None,10,Baseline,0.5883,0.8302,0.5883,0.6477,0.09
5,atis_10_subset_train,svm_glove,None,10,Baseline,0.5403,0.8625,0.5403,0.6122,8.69
6,atis_10_subset_train,lstm_glove,None,10,Baseline,0.5678,0.8367,0.5678,0.6312,37.46
7,atis_10_subset_train,bert_ktrain,None,10,Baseline,0.7993,0.8989,0.7993,0.8263,42.10
8,atis_20_subset_train,svm_tfidf,None,20,Baseline,0.7461,0.8646,0.7461,0.7845,0.19
9,atis_20_subset_train,svm_glove,None,20,Baseline,0.7238,0.8771,0.7238,0.7764,8.50


In [13]:
augmented_atis_df_result = pd.DataFrame(columns=['dataset', 'classifier', 'test_accuracy', 'model', 'subset_size', 'method'])

# Train on subset + matching augmented data
for name in ['ATIS']:
    for subset_size in subset_sizes:
        subset_path = f'NLPProject2024/datasets/{name}/sampled_subsets/ver1/{name.lower()}_{subset_size}_subset.csv'

        for lambada in lambadas:
            for llm in llms:
                if lambada == 'Lambada+Instruct' and llm == 'Mistral':
                    continue
                if lambada == 'Lambada+' and llm == 'GPT2':
                    continue
                if lambada == 'Lambada+Instruct' and llm == 'GPT2':
                    continue

                for classifier in classifiers:
                    directory_path = f'NLPProject2024/filtered_datasets/{lambada}/{llm}/{name}/{classifier}'

                    data_files = glob.glob(os.path.join(directory_path, '*.csv'))

                    for file in data_files:
                      # Hack to train with only matching augmented data
                      if f"{name.lower()}_{subset_size}_" not in file.lower():
                        continue
                      try:
                        dataset_file_name = file.replace(f'{directory_path}', "").replace('.csv', "").replace('/', "")
                        model_path = f'final_results/models/{dataset_file_name}'
                        os.makedirs(model_path, exist_ok=True)
                        print(f"Training {name} with {classifier} on {dataset_file_name}...")
                        X_train, y_train, X_test, y_test, le, X_val, y_val = load_data(subset_path,
                                                                                generated_train_path=file,
                                                                                test_path=f'NLPProject2024/datasets/{name}/{name.lower()}.test.csv',
                                                                                val_path=f'NLPProject2024/datasets/{name}/{name.lower()}.valid.csv')
                        joblib.dump(le, f'{model_path}/label_encoder.pkl')

                        trainer = ModelTrainer(classifier, glove_file, len(le.classes_))
                        train_results = trainer.train_classifier(X_train, y_train, model_path, X_test=X_test, y_test=y_test, X_val=X_val, y_val=y_val)

                        train_results["dataset"] = dataset_file_name
                        train_results["method"] = lambada
                        train_results["model"] = llm
                        train_results["subset_size"] = subset_size
                        augmented_atis_df_result = pd.concat([augmented_atis_df_result, pd.DataFrame([train_results])], ignore_index=True)
                        print(f"Finished training {name} with {classifier} on {dataset_file_name}")
                        print(train_results)
                        print("-" * 50)
                      except Exception as e:
                        print(f"Error training {name} with {classifier} on {dataset_file_name}: {e}")
                        continue

Training ATIS with svm_tfidf on Llama3_8B_ATIS_5_augmented_data_svm_tfidf_filtered...
	Full train path: NLPProject2024/datasets/ATIS/sampled_subsets/ver1/atis_5_subset.csv
	Generated train path: NLPProject2024/filtered_datasets/Lambada/Llama3/ATIS/svm_tfidf/Llama3_8B_ATIS_5_augmented_data_svm_tfidf_filtered.csv
	Test path: NLPProject2024/datasets/ATIS/atis.test.csv
Training svm_tfidf...

Validation Results:
Accuracy: 0.6707
Precision: 0.8934
Recall: 0.6707
F1 Score: 0.7354
Accuracy: 0.6930
Precision: 0.8575
Recall: 0.6930
F1 Score: 0.7358
Model saved.
Finished training ATIS with svm_tfidf on Llama3_8B_ATIS_5_augmented_data_svm_tfidf_filtered
{'classifier': 'svm_tfidf', 'test_accuracy': 0.693, 'precision': 0.8575, 'recall': 0.693, 'f1_score': 0.7358, 'training_time': 0.15, 'dataset': 'Llama3_8B_ATIS_5_augmented_data_svm_tfidf_filtered', 'method': 'Lambada', 'model': 'Llama3', 'subset_size': 5}
--------------------------------------------------
Training ATIS with svm_glove on Llama3_8B_A

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Validation Results:
Accuracy: 0.6415
Precision: 0.8506
Recall: 0.6415
F1 Score: 0.7090
Accuracy: 0.6261
Precision: 0.8045
Recall: 0.6261
F1 Score: 0.6777
Model saved.


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Finished training ATIS with svm_glove on Llama3_8B_ATIS_5_augmented_data_svm_glove_filtered
{'classifier': 'svm_glove', 'test_accuracy': 0.6261, 'precision': 0.8045, 'recall': 0.6261, 'f1_score': 0.6777, 'training_time': 8.7, 'dataset': 'Llama3_8B_ATIS_5_augmented_data_svm_glove_filtered', 'method': 'Lambada', 'model': 'Llama3', 'subset_size': 5}
--------------------------------------------------
Training ATIS with lstm_glove on Llama3_8B_ATIS_5_augmented_data_lstm_glove_filtered...
	Full train path: NLPProject2024/datasets/ATIS/sampled_subsets/ver1/atis_5_subset.csv
	Generated train path: NLPProject2024/filtered_datasets/Lambada/Llama3/ATIS/lstm_glove/Llama3_8B_ATIS_5_augmented_data_lstm_glove_filtered.csv
	Test path: NLPProject2024/datasets/ATIS/atis.test.csv
Training lstm_glove...
Epoch 1/20, Loss: 3.0638, Accuracy: 0.0392
Accuracy: 0.4288
Precision: 0.6353
Recall: 0.4288
F1 Score: 0.4963
Validation - Accuracy: 0.4288, Precision: 0.6353, Recall: 0.4288, F1 Score: 0.4963
Epoch 2/20, 

...

Epoch 18/20, Loss: 0.1484, Accuracy: 0.9731
Accuracy: 0.8679
Precision: 0.9461
Recall: 0.8679
F1 Score: 0.8972
Validation - Accuracy: 0.8679, Precision: 0.9461, Recall: 0.8679, F1 Score: 0.8972
Epoch 19/20, Loss: 0.1642, Accuracy: 0.9527
Accuracy: 0.9434
Precision: 0.9559
Recall: 0.9434
F1 Score: 0.9481
Validation - Accuracy: 0.9434, Precision: 0.9559, Recall: 0.9434, F1 Score: 0.9481


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 20/20, Loss: 0.1352, Accuracy: 0.9635
Accuracy: 0.9228
Precision: 0.9545
Recall: 0.9228
F1 Score: 0.9344
Validation - Accuracy: 0.9228, Precision: 0.9545, Recall: 0.9228, F1 Score: 0.9344


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.9057
Precision: 0.9512
Recall: 0.9057
F1 Score: 0.9216
Model and embeddings saved.
Finished training ATIS with lstm_glove on Llama_8B_lambada_plus_instruct_ATIS_100_augmented_data_lstm_glove_filtered
{'classifier': 'lstm_glove', 'test_accuracy': 0.9057, 'precision': 0.9512, 'recall': 0.9057, 'f1_score': 0.9216, 'training_time': 41.35, 'dataset': 'Llama_8B_lambada_plus_instruct_ATIS_100_augmented_data_lstm_glove_filtered', 'method': 'Lambada+Instruct', 'model': 'Llama3', 'subset_size': 100}
--------------------------------------------------


In [14]:
augmented_atis_df_result.sort_values(by=['classifier', 'test_accuracy']).head(50)

,dataset,classifier,test_accuracy,model,subset_size,method,precision,recall,f1_score,training_time
15,Llama3_8B_lambada_plus_ATIS_5_augmented_data_b...,bert_ktrain,0.7667,Llama3,5,Lambada+,0.9151,0.7667,0.8235,30.14
19,Mistral_7B_lambada_plus_ATIS_5_augmented_data_...,bert_ktrain,0.7907,Mistral,5,Lambada+,0.8879,0.7907,0.8222,34.02
23,Llama_8B_lambada_plus_instruct_ATIS_5_augmente...,bert_ktrain,0.7925,Llama3,5,Lambada+Instruct,0.9074,0.7925,0.8316,32.97
11,GPT2_ATIS_5_augmented_data_bert_ktrain_filtered,bert_ktrain,0.8182,GPT2,5,Lambada,0.8929,0.8182,0.8407,42.06
35,GPT2_ATIS_10_augmented_data_bert_ktrain_filtered,bert_ktrain,0.8439,GPT2,10,Lambada,0.9140,0.8439,0.8640,33.09
7,Mistral_7B_ATIS_5_augmented_data_bert_ktrain_f...,bert_ktrain,0.8628,Mistral,5,Lambada,0.9260,0.8628,0.8872,33.34
47,Llama_8B_lambada_plus_instruct_ATIS_10_augment...,bert_ktrain,0.8816,Llama3,10,Lambada+Instruct,0.9290,0.8816,0.8969,35.82
43,Mistral_7B_lambada_plus_ATIS_10_augmented_data...,bert_ktrain,0.8885,Mistral,10,Lambada+,0.9280,0.8885,0.8945,34.87
27,Llama3_8B_ATIS_10_augmented_data_bert_ktrain_f...,bert_ktrain,0.8902,Llama3,10,Lambada,0.9310,0.8902,0.9021,32.29
63,Llama3_8B_lambada_plus_ATIS_20_augmented_data_...,bert_ktrain,0.8902,Llama3,20,Lambada+,0.9457,0.8902,0.9083,40.00


In [15]:
augmented_trec_df_result = pd.DataFrame(columns=['dataset', 'classifier', 'test_accuracy', 'model', 'subset_size', 'method'])

# Train on subset + matching augmented data
for name in ['TREC']:
    for subset_size in subset_sizes:
        subset_path = f'NLPProject2024/datasets/{name}/sampled_subsets/ver1/{name.lower()}_{subset_size}_subset.csv'

        for lambada in lambadas:
            for llm in llms:
                if lambada == 'Lambada+Instruct' and llm == 'Mistral':
                    continue
                if lambada == 'Lambada+' and llm == 'GPT2':
                    continue
                if lambada == 'Lambada+Instruct' and llm == 'GPT2':
                    continue

                for classifier in classifiers:
                    directory_path = f'NLPProject2024/filtered_datasets/{lambada}/{llm}/{name}/{classifier}'

                    data_files = glob.glob(os.path.join(directory_path, '*.csv'))

                    for file in data_files:
                      # Hack to train with only matching augmented data
                      if f"{name.lower()}_{subset_size}_" not in file.lower():
                        continue
                      try:
                        dataset_file_name = file.replace(f'{directory_path}', "").replace('.csv', "").replace('/', "")
                        model_path = f'final_results/models/{dataset_file_name}'
                        os.makedirs(model_path, exist_ok=True)
                        print(f"Training {name} with {classifier} on {dataset_file_name}...")
                        X_train, y_train, X_test, y_test, le, X_val, y_val = load_data(subset_path,
                                                                                generated_train_path=file,
                                                                                test_path=f'NLPProject2024/datasets/{name}/{name.lower()}.test.csv',
                                                                                val_path=f'NLPProject2024/datasets/{name}/{name.lower()}.valid.csv')
                        joblib.dump(le, f'{model_path}/label_encoder.pkl')

                        trainer = ModelTrainer(classifier, glove_file, len(le.classes_))
                        train_results = trainer.train_classifier(X_train, y_train, model_path, X_test=X_test, y_test=y_test, X_val=X_val, y_val=y_val)

                        train_results["dataset"] = dataset_file_name
                        train_results["method"] = lambada
                        train_results["model"] = llm
                        train_results["subset_size"] = subset_size
                        augmented_trec_df_result = pd.concat([augmented_trec_df_result, pd.DataFrame([train_results])], ignore_index=True)
                        print(f"Finished training {name} with {classifier} on {dataset_file_name}")
                        print(train_results)
                        print("-" * 50)
                      except Exception as e:
                        print(f"Error training {name} with {classifier} on {dataset_file_name}: {e}")
                        continue

Training TREC with svm_tfidf on Llama3_8B_TREC_5_augmented_data_svm_tfidf_filtered...
	Full train path: NLPProject2024/datasets/TREC/sampled_subsets/ver1/trec_5_subset.csv
	Generated train path: NLPProject2024/filtered_datasets/Lambada/Llama3/TREC/svm_tfidf/Llama3_8B_TREC_5_augmented_data_svm_tfidf_filtered.csv
	Test path: NLPProject2024/datasets/TREC/trec.test.csv
Training svm_tfidf...

Validation Results:
Accuracy: 0.4639
Precision: 0.6540
Recall: 0.4639
F1 Score: 0.4726


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.4956
Precision: 0.7240
Recall: 0.4956
F1 Score: 0.5090
Model saved.
Finished training TREC with svm_tfidf on Llama3_8B_TREC_5_augmented_data_svm_tfidf_filtered
{'classifier': 'svm_tfidf', 'test_accuracy': 0.4956, 'precision': 0.724, 'recall': 0.4956, 'f1_score': 0.509, 'training_time': 1.06, 'dataset': 'Llama3_8B_TREC_5_augmented_data_svm_tfidf_filtered', 'method': 'Lambada', 'model': 'Llama3', 'subset_size': 5}
--------------------------------------------------
Training TREC with svm_glove on Llama3_8B_TREC_5_augmented_data_svm_glove_filtered...
	Full train path: NLPProject2024/datasets/TREC/sampled_subsets/ver1/trec_5_subset.csv
	Generated train path: NLPProject2024/filtered_datasets/Lambada/Llama3/TREC/svm_glove/Llama3_8B_TREC_5_augmented_data_svm_glove_filtered.csv
	Test path: NLPProject2024/datasets/TREC/trec.test.csv
Training svm_glove...

Validation Results:
Accuracy: 0.3273
Precision: 0.4691
Recall: 0.3273
F1 Score: 0.3350


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.3377
Precision: 0.4810
Recall: 0.3377
F1 Score: 0.3504
Model saved.
Finished training TREC with svm_glove on Llama3_8B_TREC_5_augmented_data_svm_glove_filtered
{'classifier': 'svm_glove', 'test_accuracy': 0.3377, 'precision': 0.481, 'recall': 0.3377, 'f1_score': 0.3504, 'training_time': 9.39, 'dataset': 'Llama3_8B_TREC_5_augmented_data_svm_glove_filtered', 'method': 'Lambada', 'model': 'Llama3', 'subset_size': 5}
--------------------------------------------------
Training TREC with lstm_glove on Llama3_8B_TREC_5_augmented_data_lstm_glove_filtered...
	Full train path: NLPProject2024/datasets/TREC/sampled_subsets/ver1/trec_5_subset.csv
	Generated train path: NLPProject2024/filtered_datasets/Lambada/Llama3/TREC/lstm_glove/Llama3_8B_TREC_5_augmented_data_lstm_glove_filtered.csv
	Test path: NLPProject2024/datasets/TREC/trec.test.csv
Training lstm_glove...


...

Accuracy: 0.7179
Precision: 0.7662
Recall: 0.7179
F1 Score: 0.7141
Validation - Accuracy: 0.7179, Precision: 0.7662, Recall: 0.7179, F1 Score: 0.7141
Epoch 7/20, Loss: 0.7840, Accuracy: 0.7963
Accuracy: 0.7041
Precision: 0.7698
Recall: 0.7041
F1 Score: 0.7027
Validation - Accuracy: 0.7041, Precision: 0.7698, Recall: 0.7041, F1 Score: 0.7027
Epoch 8/20, Loss: 0.7002, Accuracy: 0.8163


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.7392
Precision: 0.7863
Recall: 0.7392
F1 Score: 0.7374
Validation - Accuracy: 0.7392, Precision: 0.7863, Recall: 0.7392, F1 Score: 0.7374
Epoch 9/20, Loss: 0.6310, Accuracy: 0.8324
Accuracy: 0.7561
Precision: 0.7997
Recall: 0.7561
F1 Score: 0.7608
Validation - Accuracy: 0.7561, Precision: 0.7997, Recall: 0.7561, F1 Score: 0.7608
Epoch 10/20, Loss: 0.5439, Accuracy: 0.8530
Accuracy: 0.7567
Precision: 0.8045
Recall: 0.7567
F1 Score: 0.7605
Validation - Accuracy: 0.7567, Precision: 0.8045, Recall: 0.7567, F1 Score: 0.7605
Epoch 11/20, Loss: 0.4998, Accuracy: 0.8607
Accuracy: 0.7599
Precision: 0.8100
Recall: 0.7599
F1 Score: 0.7644
Validation - Accuracy: 0.7599, Precision: 0.8100, Recall: 0.7599, F1 Score: 0.7644
Epoch 12/20, Loss: 0.4478, Accuracy: 0.8737
Accuracy: 0.7668
Precision: 0.8118
Recall: 0.7668
F1 Score: 0.7666
Validation - Accuracy: 0.7668, Precision: 0.8118, Recall: 0.7668, F1 Score: 0.7666
Epoch 13/20, Loss: 0.4140, Accuracy: 0.8907


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.7705
Precision: 0.8208
Recall: 0.7705
F1 Score: 0.7763
Validation - Accuracy: 0.7705, Precision: 0.8208, Recall: 0.7705, F1 Score: 0.7763
Epoch 14/20, Loss: 0.3787, Accuracy: 0.8935
Accuracy: 0.7712
Precision: 0.8226
Recall: 0.7712
F1 Score: 0.7746
Validation - Accuracy: 0.7712, Precision: 0.8226, Recall: 0.7712, F1 Score: 0.7746
Epoch 15/20, Loss: 0.3266, Accuracy: 0.9097
Accuracy: 0.7743
Precision: 0.8177
Recall: 0.7743
F1 Score: 0.7780
Validation - Accuracy: 0.7743, Precision: 0.8177, Recall: 0.7743, F1 Score: 0.7780
Epoch 16/20, Loss: 0.3073, Accuracy: 0.9156
Accuracy: 0.7843
Precision: 0.8282
Recall: 0.7843
F1 Score: 0.7832
Validation - Accuracy: 0.7843, Precision: 0.8282, Recall: 0.7843, F1 Score: 0.7832
Epoch 17/20, Loss: 0.2716, Accuracy: 0.9261
Accuracy: 0.7950
Precision: 0.8230
Recall: 0.7950
F1 Score: 0.7949
Validation - Accuracy: 0.7950, Precision: 0.8230, Recall: 0.7950, F1 Score: 0.7949
Epoch 18/20, Loss: 0.2599, Accuracy: 0.9289
Accuracy: 0.7749
Precision: 0.

In [16]:
augmented_trec_df_result.sort_values(by=['classifier', 'test_accuracy']).head(40)

,dataset,classifier,test_accuracy,model,subset_size,method,precision,recall,f1_score,training_time
11,Llama3_8B_lambada_plus_TREC_5_augmented_data_l...,lstm_glove,0.4154,Llama3,5,Lambada+,0.6742,0.4154,0.4233,44.21
8,GPT2_TREC_5_augmented_data_lstm_glove_filtered,lstm_glove,0.4204,GPT2,5,Lambada,0.5272,0.4204,0.4253,43.81
5,Mistral_7B_TREC_5_augmented_data_lstm_glove_fi...,lstm_glove,0.4411,Mistral,5,Lambada,0.5466,0.4411,0.4525,44.26
2,Llama3_8B_TREC_5_augmented_data_lstm_glove_fil...,lstm_glove,0.4706,Llama3,5,Lambada,0.6037,0.4706,0.4849,43.99
17,Llama_8B_lambada_plus_instruct_TREC_5_augmente...,lstm_glove,0.4768,Llama3,5,Lambada+Instruct,0.6233,0.4768,0.4799,44.39
14,Mistral_7B_lambada_plus_TREC_5_augmented_data_...,lstm_glove,0.4956,Mistral,5,Lambada+,0.6046,0.4956,0.5077,45.08
26,GPT2_TREC_10_augmented_data_lstm_glove_filtered,lstm_glove,0.5320,GPT2,10,Lambada,0.6549,0.5320,0.5467,44.70
35,Llama_8B_lambada_plus_instruct_TREC_10_augment...,lstm_glove,0.5445,Llama3,10,Lambada+Instruct,0.6788,0.5445,0.5610,46.72
23,Mistral_7B_TREC_10_augmented_data_lstm_glove_f...,lstm_glove,0.5608,Mistral,10,Lambada,0.6678,0.5608,0.5678,45.13
32,Mistral_7B_lambada_plus_TREC_10_augmented_data...,lstm_glove,0.5739,Mistral,10,Lambada+,0.6646,0.5739,0.5787,45.07


In [17]:
df_result = pd.concat([full_df_result, subsets_df_result, augmented_atis_df_result, augmented_trec_df_result], ignore_index=True)
df_result.to_csv("final_results/test_results.csv")

In [18]:
df_result.head(50)

,dataset,model,test_accuracy,precision,recall,f1_score,classifier,training_time,subset_size,method
0,atis_full,NaN,0.9588,0.9516,0.9588,0.9532,svm_tfidf,6.85,NaN,NaN
1,atis_full,NaN,0.9280,0.9121,0.9280,0.9170,svm_glove,11.49,NaN,NaN
2,atis_full,NaN,0.9794,0.9769,0.9794,0.9776,lstm_glove,62.08,NaN,NaN
3,atis_full,NaN,0.9828,0.9808,0.9828,0.9814,bert_ktrain,153.83,NaN,NaN
4,trec_full,NaN,0.9599,0.9611,0.9599,0.9598,svm_tfidf,81.94,NaN,NaN
5,trec_full,NaN,0.8177,0.8224,0.8177,0.8168,svm_glove,40.66,NaN,NaN
6,trec_full,NaN,0.9599,0.9616,0.9599,0.9591,lstm_glove,112.23,NaN,NaN
7,trec_full,NaN,0.9862,0.9870,0.9862,0.9862,bert_ktrain,827.74,NaN,NaN
8,atis_5_subset_train,None,0.6055,0.8235,0.6055,0.6602,svm_tfidf,0.09,5,Baseline
9,atis_5_subset_train,None,0.5815,0.7998,0.5815,0.6436,svm_glove,8.55,5,Baseline
